## Preprocess and explore datasets

## TODOs

- Try fb prophet for time series - only
- Explore spatial econometric methods
- Slice according to percentiles/quartiles to find which can give best results

In [2]:
import pandas as pd
import folium as fol
import geohash2 as gh
import seaborn as sns
import geopandas as gpd
from folium import plugins
import matplotlib.pyplot as plt
from shapely.geometry import Point
%matplotlib inline

In [3]:
ds_path = r'C:\Users\jose.luigi.s.torres\Desktop\aiforsea\traffic_management\data\training.csv'

df = pd.read_csv(ds_path)

In [4]:
######### TEST ON SUBSET FIRST #########
subset_test = pd.DataFrame(df.loc[:100,])
decoded = subset_test.geohash6.apply(lambda x: gh.decode_exactly(x))
subset_test['lat'] = decoded.apply(lambda x: x[0])
subset_test['long'] = decoded.apply(lambda x: x[1])

subset_test['hour'] = subset_test.timestamp.str.split(':').str[0]
subset_test['min'] = subset_test.timestamp.str.split(':').str[1]

geom_test = [Point(xy) for xy in zip(subset_test.lat.astype('float'),subset_test.long.astype('float'))]
gdf_test = gpd.GeoDataFrame(subset_test,geometry=geom_test)

subset_test.drop(['geohash6'],axis=1,inplace=True)

In [5]:
# preprocess coordinates
decoded = df.geohash6.apply(lambda x: gh.decode_exactly(x))
df['lat'] = decoded.apply(lambda x: x[0])
df['long'] = decoded.apply(lambda x: x[1])

# convert to geodataframe
geom = [Point(xy) for xy in zip(df.lat.astype('float'),df.long.astype('float'))]
gdf = gpd.GeoDataFrame(df,geometry=geom)

In [6]:
gdf['hour'] = gdf.timestamp.str.split(':').str[0]
gdf['min'] = gdf.timestamp.str.split(':').str[1]

gdf['hour'] = gdf['hour'].astype('int')
gdf['min'] = gdf['min'].astype('int')


gdf.drop(['geohash6',],axis=1,inplace=True)

In [8]:
gdf.query("day==1")

,day,timestamp,demand,lat,long,geometry,hour,min
5,1,12:15,0.023843,-5.336609,90.609741,POINT (-5.33660888671875 90.6097412109375),12,15
172,1,21:30,0.090481,-5.358582,90.642700,POINT (-5.35858154296875 90.6427001953125),21,30
302,1,0:45,0.060016,-5.342102,90.730591,POINT (-5.34210205078125 90.7305908203125),0,45
402,1,15:30,0.035535,-5.358582,90.686646,POINT (-5.35858154296875 90.6866455078125),15,30
408,1,6:30,0.015310,-5.468445,90.664673,POINT (-5.46844482421875 90.6646728515625),6,30
491,1,2:15,0.002061,-5.435486,90.807495,POINT (-5.43548583984375 90.8074951171875),2,15
563,1,8:0,0.087680,-5.347595,90.840454,POINT (-5.34759521484375 90.8404541015625),8,0
687,1,16:30,0.012704,-5.331116,90.631714,POINT (-5.33111572265625 90.6317138671875),16,30
739,1,12:0,0.107851,-5.336609,90.697632,POINT (-5.33660888671875 90.6976318359375),12,0
844,1,13:45,0.078627,-5.446472,90.686646,POINT (-5.44647216796875 90.6866455078125),13,45


In [1]:
import pysal

C:\Users\jose.luigi.s.torres\.conda\envs\aiforsea\lib\site-packages\pysal\__init__.py:65: VisibleDeprecationWarning: PySAL's API will be changed on 2018-12-31. The last release made with this API is version 1.14.4. A preview of the next API version is provided in the `pysalnext` package. The API changes and a guide on how to change imports is provided at https://migrating.pysal.org
  ), VisibleDeprecationWarning)
